In [ ]:
import pandas as pd
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv1D, SpatialDropout1D, Add, Activation, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Configuration
tf.random.set_seed(42)
np.random.seed(42)

# ==============================================================================
# 1. PRÉPARATION DES DONNÉES EN SÉQUENCES
# ==============================================================================
print("Chargement et structuration des données...")
data = pd.read_csv('data/processed_data.csv')
data['date'] = pd.to_datetime(data['date'])

# Log-transform cible
data.loc[data['is_train'] == 1, 'sales'] = np.log1p(data.loc[data['is_train'] == 1, 'sales'])

# Tri CRITIQUE : Il faut que les données soient ordonnées temporellement par série
data = data.sort_values(['store_nbr', 'family', 'date']).reset_index(drop=True)

# Définition des features
# On enlève 'date', 'id', 'is_train' et 'sales' (cible) des features d'entrée
EXCLUDED = ['id', 'date', 'sales', 'is_train']
FEATURES = [c for c in data.columns if c not in EXCLUDED]

# Encodage et Scaling (Rapide)
print("Encodage et Scaling...")
cat_cols = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster', 'month', 'dayofweek', 'is_holiday']
for col in cat_cols:
    if col in data.columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col].astype(str))

scaler = StandardScaler()
# On scale uniquement les colonnes numériques (pas les catégories encodées)
num_cols = [c for c in FEATURES if c not in cat_cols]
data[num_cols] = scaler.fit_transform(data[num_cols])
data[FEATURES] = data[FEATURES].fillna(0)

# ==============================================================================
# 2. CRÉATION DU CUBE 3D (Séries, Temps, Features)
# ==============================================================================
print("Transformation en Cube 3D...")

# Constantes
N_STORES = 54
N_FAMILIES = 33
N_SERIES = N_STORES * N_FAMILIES # 1782 séries temporelles
N_DAYS = len(data) // N_SERIES   # Nombre de jours total
N_FEATURES = len(FEATURES)

# Vérification d'intégrité
assert len(data) % N_SERIES == 0, "Erreur: Le nombre de lignes n'est pas un multiple parfait de (Stores * Families)"

# RESHAPE MAGIQUE : (1782, Jours, Features)
# X_all contient toutes les features pour toutes les séries
X_all = data[FEATURES].values.reshape(N_SERIES, N_DAYS, N_FEATURES)
# y_all contient toutes les ventes (cibles)
y_all = data['sales'].values.reshape(N_SERIES, N_DAYS, 1)

# Séparation Train / Test (basée sur le temps)
# Le test set Kaggle est les 16 derniers jours
HORIZON = 16
LOOKBACK = 60 # On regarde 60 jours en arrière pour prédire

# Index de séparation
train_end_idx = N_DAYS - HORIZON # Fin du training set (exclut le test Kaggle)

# Données d'entraînement (Tout sauf les 16 derniers jours qui sont le test Kaggle)
X_train_full = X_all[:, :train_end_idx, :]
y_train_full = y_all[:, :train_end_idx, :]

print(f"Forme du Cube 3D Train : {X_train_full.shape}")

# ==============================================================================
# 3. GÉNÉRATEUR DE SÉQUENCES (SLIDING WINDOW)
# ==============================================================================
# Pour entraîner, on ne peut pas juste donner le cube. Il faut créer des paires (X, y)
# X: Fenêtre de 60 jours
# y: Les 16 jours suivants (Stratégie Directe Multi-Output)

def create_dataset(X_3d, y_3d, lookback, horizon):
    X_seq, y_seq = [], []
    
    # On itère sur le temps pour créer des fenêtres
    # On prend un pas de 16 pour éviter trop de redondance et aller vite (stride)
    # Pour plus de précision, mettre stride=1 (mais beaucoup plus lourd en RAM)
    n_timesteps = X_3d.shape[1]
    
    # On s'arrête avant la fin pour avoir la place pour l'horizon
    for t in range(lookback, n_timesteps - horizon + 1, 8): # Stride de 8 jours
        # Input : t-lookback à t
        window_x = X_3d[:, t-lookback:t, :] 
        # Output : t à t+horizon
        window_y = y_3d[:, t:t+horizon, 0]
        
        # On empile les 1782 séries pour ce pas de temps
        X_seq.append(window_x)
        y_seq.append(window_y)
        
    return np.vstack(X_seq), np.vstack(y_seq)

print("Génération des fenêtres glissantes (Patience...)...")
# On garde les 16 derniers jours du train_full pour la validation
val_split_idx = X_train_full.shape[1] - HORIZON

X_train_data = X_train_full[:, :val_split_idx, :]
y_train_data = y_train_full[:, :val_split_idx, :]

X_val_data = X_train_full[:, val_split_idx-LOOKBACK:, :] # On inclut le lookback pour avoir le contexte
y_val_data = y_train_full[:, val_split_idx:, :] # Cible de validation (ce sont les 15 derniers jours connus)

# Création des datasets numpy
X_train, y_train = create_dataset(X_train_data, y_train_data, LOOKBACK, HORIZON)

# Pour la validation, on prend juste la dernière fenêtre disponible
X_val = X_val_data[:, :LOOKBACK, :]
y_val = y_val_data[:, :HORIZON, 0] # (1782, 16)

print(f"Input Train Shape: {X_train.shape}") # (Samples, 60, Features)
print(f"Output Train Shape: {y_train.shape}") # (Samples, 16)

# ==============================================================================
# 4. MODÈLE TCN (VRAIE SÉQUENCE)
# ==============================================================================
def build_tcn_classic(input_shape, output_horizon):
    inputs = Input(shape=input_shape)
    
    x = inputs
    
    # Bloc TCN : Dilated Convolutions
    # Dilation 1, 2, 4, 8, 16 permet de voir 1 + 2 + 4 + 8 + 16 = 31 jours en arrière avec peu de couches
    for dilation in [1, 2, 4, 8, 16]:
        # Residual connection setup
        shortcut = x if x.shape[-1] == 64 else Conv1D(64, 1)(x)
        
        # Conv1D Dilatée
        x = Conv1D(filters=64, 
                   kernel_size=3, 
                   dilation_rate=dilation, 
                   padding='causal', # IMPORTANT : Causal pour ne pas voir le futur
                   activation='relu')(x)
        x = SpatialDropout1D(0.1)(x)
        
        # Skip connection
        x = Add()([x, shortcut])
        x = Activation('relu')(x)
    
    # On ne garde que la dernière étape de temps (GlobalAveragePooling ou Slicing)
    # Ici on utilise GlobalAverage pour résumer toute la fenêtre
    x = GlobalAveragePooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    
    # Output layer : Prédit 16 jours d'un coup
    outputs = Dense(output_horizon, activation='linear')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

print("\nConstruction du TCN...")
model = build_tcn_classic((LOOKBACK, N_FEATURES), HORIZON)
model.summary()

# ==============================================================================
# 5. ENTRAÎNEMENT
# ==============================================================================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=128,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
    ]
)

# ==============================================================================
# 6. PRÉDICTION FINALE (SUR LE TEST SET KAGGLE)
# ==============================================================================
print("\nPrédiction finale...")

# Pour prédire le futur (Test Kaggle), on prend les 60 derniers jours connus de TOUT le dataset
last_window = X_all[:, -LOOKBACK:, :] # Shape (1782, 60, Features)

# Prédiction (Log scale)
preds_log = model.predict(last_window) # Shape (1782, 16)

# Inverse Log
preds = np.expm1(preds_log)
preds[preds < 0] = 0

# Remise en forme pour le fichier de soumission
# Les prédictions sont de forme (1782 séries, 16 jours)
# Il faut les aplatir dans l'ordre : Série 1 (J1..J16), Série 2 (J1..J16)...
# ATTENTION : Il faut s'assurer que l'ordre correspond à celui du fichier test.csv
# Le fichier test.csv est trié par Date puis Store puis Family ? Non, souvent Store/Family/Date.

# On va reconstruire le DataFrame de soumission proprement
# On récupère les IDs du test set original
test_df = data[data['is_train'] == 0].copy()
# On s'assure que test_df est trié exactement comme notre X_all (Store, Family, Date)
test_df = test_df.sort_values(['store_nbr', 'family', 'date'])

# Aplatissement des prédictions : (1782, 16) -> (1782 * 16,)
# .ravel() aplatit ligne par ligne (Série 1 J1, Série 1 J2 ... Série 1 J16, Série 2 J1 ...)
# C'est exactement ce qu'on veut si test_df est trié par (Store, Family) puis Date.
flat_preds = preds.ravel()

test_df['sales'] = flat_preds

# On garde juste id et sales, et on remet dans l'ordre des IDs (pour Kaggle)
submission = test_df[['id', 'sales']].sort_values('id')
submission.to_csv('submission_tcn_classic_3d.csv', index=False)

print("Terminé ! Fichier 'submission_tcn_classic_3d.csv' généré.")